In [ ]:
#Import the libraries 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly

In [2]:
##Cleaning Table 1##

#Read Table 1 on Excel without the first 14 rows

data_table1 = pd.read_excel('UN_MigrantStockTotal_2015.xlsx', 'Table 1', skiprows=14)

##Problem: The column names are values (years) and not variable names 
#tidy data principle #1: Column names need to be informative, variable names and not values

# The years are in the first row. To fix that problem, we have to change it into columns with their corresponding values. 
data_table1.columns = data_table1.iloc[0]

#Manually re-write the column names and organize based off of desired column names 

data_table1.columns = ['Sort\norder', 'Major area, region, country or area of destination', 'Notes', 'Country code', 'Type of data (a)',
'1990MF', '1995MF', '2000MF', '2005MF', '2010MF', '2015MF', '1990M', '1995M', '2000M', '2005M', '2010M', '2015M', '1990F', '1995F', '2000F', '2005F', '2010F', '2015F']


#Remove first row 

data_table1 = data_table1.iloc[1: , :]

#Create a column name called demographic that has the year+gender values

data_table1 = data_table1.melt(id_vars=['Sort\norder', 'Major area, region, country or area of destination', 'Notes', 'Country code', 'Type of data (a)'], var_name=["demographic"],value_name="International Migrant Stock at Mid-Year") 

##Problem: There are multiple variables (gender and year) stored in one column (demographic) 
#Tidy data principle 2: Each column needs to consist of one and only one variable
#Solution: Split the demographic column into gender and years 

data_table1 = (data_table1.assign(Year = lambda x: x.demographic.str[0:4].astype(str), Gender = lambda x: x.demographic.str[4:].astype(str)).drop("demographic",axis=1))

#Styling the dataset: Changing MF to Both sexes, M to male, F to females.Change Sort\norder column name to Sort Order

data_table1 = (data_table1.replace(to_replace =["MF","M","F"],value =["Both Sexes","Male","Female"]))
data_table1 = data_table1.rename(columns={'Sort\norder': 'Sort Order'})    

#Drop the empty cells that contain value = .. 

data_table1['International Migrant Stock at Mid-Year'].replace('..', np.nan, inplace = True)  
data_table1.dropna(subset=['International Migrant Stock at Mid-Year'], inplace=True)


##Problem: The first 6 rows contain values whose country code does not exist in the annex. These rows also are sums of the other rows and do not belong under the same column. 
#Tidy data principle #4: each table column needs to have a singular data type 
#Solution: We created a seperate table that holds an executive summary of the International Migrant Stock at Mid-Year

#Create a new dataframe containing desired columns

table1_executive_summary = data_table1[['Major area, region, country or area of destination', 'Notes', 'International Migrant Stock at Mid-Year', 'Year', 'Gender']].copy()

#table1_ES has only the values "WORLD", "Developed regions", "Developing regions", "Least developed countries", "Least developed regions", and "Sub-Saharan Africa"

values=["WORLD","Developed regions", "Developing regions", "Least developed countries", "Less developed regions excluding least developed countries", "Sub-Saharan Africa"]
table1_ES = table1_executive_summary[table1_executive_summary["Major area, region, country or area of destination"].isin(values)] 

#Rename 
table1_ES = table1_ES.rename({'Major area, region, country or area of destination': 'Location'}, axis=1) 

table1_ES = table1_ES.reset_index(drop=True)

#Fix table 1 and remove all values in the values list
new_table1 = data_table1[~data_table1["Major area, region, country or area of destination"].isin(values)] 
new_table1 = new_table1.reset_index(drop=True)


#Problem: Major area, region, country or area of destination has multiple variables in the column and different variables in the rows. 
#tidy data principle #3: variables need to be in cells, not rows and columns
#Solution: Add a new column called "Category" that seperates the major area, region, country/area of destination 

#Create list
list_major_area_num = [7, 71, 127, 180, 238]
list_region_num = [8, 29, 39, 47, 53, 72, 78, 86, 98, 108, 128, 139, 153, 170, 181, 208, 217, 232, 239, 242, 248, 256]

# function to assign values in the category values of the new column 
def category(df):
    if df['Sort Order'] in list_major_area_num:
        return 'Major Area'
    elif df['Sort Order'] in list_region_num:
        return 'Region'
    return 'Country/Area'

new_table1 = new_table1.rename(columns={'Major area, region, country or area of destination': 'Location', 'Country code': 'Country Code'})

#change the float types into ints
new_table1['Sort Order'] = new_table1['Sort Order'].astype(int)
new_table1['Country Code'] = new_table1['Country Code'].astype(int)

#Add category column
new_table1['Category'] = new_table1.apply(category, axis = 1)


#reorder the columns so that it follows order: sort order, country code..
new_table1 = new_table1[['Sort Order', 'Location', 'Category','Country Code', 'Notes', 'Type of data (a)', 'International Migrant Stock at Mid-Year', 'Year', 'Gender']]

#view new_table1
table1_ES.head()

Location Notes  \
0                                              WORLD   NaN   
1                                  Developed regions   (b)   
2                                 Developing regions   (c)   
3                          Least developed countries   (d)   
4  Less developed regions excluding least develop...   NaN   

   International Migrant Stock at Mid-Year  Year      Gender  
0                              152563212.0  1990  Both Sexes  
1                               82378628.0  1990  Both Sexes  
2                               70184584.0  1990  Both Sexes  
3                               11075966.0  1990  Both Sexes  
4                               59105261.0  1990  Both Sexes

In [7]:
##Cleaning Table 2##

#Read Table 2 on Excel without the first 14 rows
data_table2 = pd.read_excel('UN_MigrantStockTotal_2015.xlsx', 'Table 2', skiprows=14)

##Problem: The column names are values (years) and not variable names 
#tidy data principle #1: Column names need to be informative, variable names and not values

# The years are in the first row. To fix that problem, we have to change it into columns with their corresponding values. 
data_table2.columns = data_table2.iloc[0]

# Manually re-write the column names and organize based off of desired names
data_table2.columns = ['Sort order', 'Major area, region, country or area of destination', 'Notes', 'Country code',
'1990MF', '1995MF', '2000MF', '2005MF', '2010MF', '2015MF', '1990M', '1995M', '2000M', '2005M', '2010M', '2015M', '1990F', '1995F', '2000F', '2005F', '2010F', '2015F']

#Remove first row 
data_table2 = data_table2.iloc[1: , :]

#Create a column name called demographic that has the year+gender values
data_table2 = data_table2.melt(id_vars=['Sort order', 'Major area, region, country or area of destination', 'Notes', 'Country code'], var_name=["demographic"],value_name="Total Population at Mid-Year") 


##Problem: There are multiple variables (gender and year) stored in one column (demographic) 
#Tidy data principle 2: Each column needs to consist of one and only one variable
#Solution: Split the demographic column into gender and years 
data_table2=(data_table2.assign(Year = lambda x: x.demographic.str[0:4].astype(str), Gender = lambda x: x.demographic.str[4:].astype(str)).drop("demographic",axis=1))

#Styling the dataset: Changing MF to Both sexes, M to male, F to females. Change Sort order column name to Sort Order
data_table2=(data_table2.replace(to_replace =["MF","M","F"],value =["Both Sexes","Male","Female"]))
data_table2 = data_table2.rename(columns={'Sort order': 'Sort Order'})        

#Drop the empty cells that contain value = .. 
data_table2['Total Population at Mid-Year'].replace('..', np.nan, inplace = True)  
data_table2.dropna(subset=['Total Population at Mid-Year'], inplace=True)

##Problem: The first 6 rows contain values whose country code does not exist in the annex. These rows also are sums of the other rows and do not belong under the same column. 
#Tidy data principle #4: each table column needs to have a singular data type 
#Solution: We created a seperate table that holds an executive summary of the Total Population at Mid-Year 


#Create a new dataframe containing desired columns
table2_executive_summary = data_table2[['Major area, region, country or area of destination', 'Notes', 'Total Population at Mid-Year', 'Year', 'Gender']].copy()

#table2_ES has only the values "WORLD", "Developed regions", "Developing regions", "Least developed countries", "Least developed regions", and "Sub-Saharan Africa"
values=["WORLD","Developed regions", "Developing regions", "Least developed countries", "Less developed regions excluding least developed countries", "Sub-Saharan Africa"]
new_table2_ES = table2_executive_summary[table2_executive_summary["Major area, region, country or area of destination"].isin(values)] 

new_table2_ES = new_table2_ES.reset_index(drop=True)

#Rename 
new_table2_ES = new_table2_ES.rename({'Major area, region, country or area of destination': 'Location'}, axis=1) 

#Fix table 2 and remove all values in the values list
new_table2 = data_table2[~data_table2["Major area, region, country or area of destination"].isin(values)] 
new_table2 = new_table2.reset_index(drop=True)


#Problem: Major area, region, country or area of destination has multiple variables in the column and different variables in the rows. 
#tidy data principle #3: variables need to be in cells, not rows and columns
#Solution: Add a new column called "Category" that seperates the major area, region, country/area of destination 


# function to assign values in the category values of the new column 
def category(df):
    if df['Sort Order'] in list_major_area_num:
        return 'Major Area'
    elif df['Sort Order'] in list_region_num:
        return 'Region'
    return 'Country/Area'

#Rename 
new_table2 = new_table2.rename(columns={'Major area, region, country or area of destination': 'Location', 'Country code': 'Country Code'})

#change the float types into ints
new_table2['Sort Order'] = new_table2['Sort Order'].astype(int)
new_table2['Country Code'] = new_table2['Country Code'].astype(int)

#Add category column
new_table2['Category'] = new_table2.apply(category, axis = 1)


#reorder the columns so that it follows order: sort order, country code..
new_table2 = new_table2[['Sort Order', 'Location', 'Category','Country Code', 'Notes', 'Total Population at Mid-Year', 'Year', 'Gender']]

#view new_table2
new_table2_ES.head()


Location Notes  \
0                                              WORLD   NaN   
1                                  Developed regions   (b)   
2                                 Developing regions   (c)   
3                          Least developed countries   (d)   
4  Less developed regions excluding least develop...   NaN   

   Total Population at Mid-Year  Year      Gender  
0                   5309667.699  1990  Both Sexes  
1                   1144463.062  1990  Both Sexes  
2                   4165204.637  1990  Both Sexes  
3                    510057.629  1990  Both Sexes  
4                   3655147.008  1990  Both Sexes

In [10]:
##Combine Table 1 & Table 2
#tidy data principle 5: a single observational units must be in 1 table

#Combine new_table1, new_table2
midyear_data = pd.merge(new_table1, new_table2)
midyear_data = midyear_data[['Sort Order', 'Location', 'Category', 'Country Code', 'Notes', 'Type of data (a)', 'Year', 'Gender', 'International Migrant Stock at Mid-Year', 'Total Population at Mid-Year']]

#Combine table1_ES and new_table2_ES
ES_table12 = pd.merge(table1_ES, new_table2_ES)
ES_table12 = ES_table12[['Location', 'Notes', 'Year', 'Gender', 'International Migrant Stock at Mid-Year', 'Total Population at Mid-Year']]



ES_table12.head()

Location Notes  Year      Gender  \
0                                              WORLD   NaN  1990  Both Sexes   
1                                  Developed regions   (b)  1990  Both Sexes   
2                                 Developing regions   (c)  1990  Both Sexes   
3                          Least developed countries   (d)  1990  Both Sexes   
4  Less developed regions excluding least develop...   NaN  1990  Both Sexes   

   International Migrant Stock at Mid-Year  Total Population at Mid-Year  
0                              152563212.0                   5309667.699  
1                               82378628.0                   1144463.062  
2                               70184584.0                   4165204.637  
3                               11075966.0                    510057.629  
4                               59105261.0                   3655147.008

In [15]:
##Cleaning Table 3##

#Read Table 3 on Excel without the first 14 rows
data_table3 = pd.read_excel('UN_MigrantStockTotal_2015.xlsx', 'Table 3', skiprows=14)


##Problem: The column names are values (years) and not variable names 
#tidy data principle #1: Column names need to be informative, variable names and not values


# The years are in the first row. To fix that problem, we have to change it into columns with their corresponding values. 
data_table3.columns = data_table3.iloc[0]

# Manually re-write the column names and organize based off of desired names
data_table3.columns = ['Sort order', 'Major area, region, country or area of destination', 'Notes', 'Country code', 'Type of data (a)',
'1990MF', '1995MF', '2000MF', '2005MF', '2010MF', '2015MF', '1990M', '1995M', '2000M', '2005M', '2010M', '2015M', '1990F', '1995F', '2000F', '2005F', '2010F', '2015F']

#Remove the first row                        
data_table3 = data_table3.iloc[1: , :]

#Create a column name called demographic that has the year+gender values
data_table3 = data_table3.melt(id_vars=['Sort order', 'Major area, region, country or area of destination', 'Notes', 'Country code', 'Type of data (a)'], var_name=["demographic"],value_name="International Migrant Stock as a Percentage of Total Population") 

##Problem: There are multiple variables stored in one column 
#Tidy data principle 2: Each column needs to consist of one and only one variable
#Solution: Split the demographic column into gender and years 
data_table3=(data_table3.assign(Year = lambda x: x.demographic.str[0:4].astype(str), Gender = lambda x: x.demographic.str[4:].astype(str)).drop("demographic",axis=1))

#Styling the dataset: Changing MF to Both sexes, M to male, F to females. Change Sort order column name to Sort Order 
data_table3 = (data_table3.replace(to_replace =["MF","M","F"],value =["Both Sexes","Male","Female"]))
data_table3 = data_table3.rename(columns={'Sort order': 'Sort Order'})        


#Drop the empty cells that contain value = .. 
data_table3['International Migrant Stock as a Percentage of Total Population'].replace('..', np.nan, inplace = True)  
data_table3.dropna(subset=['International Migrant Stock as a Percentage of Total Population'], inplace=True)

##Problem: The first 6 rows contain values whose country code does not exist in the annex. These rows also are sums of the other rows and do not belong under the same column. 
#Tidy data principle #4: each table column needs to have a singular data type 
#Solution: We created a seperate table that holds an executive summary of the International Migrant Stock as a Percentage of Total Population

#Create a new dataframe containing desired columns
table3_executive_summary = data_table3[['Major area, region, country or area of destination', 'Notes', 'International Migrant Stock as a Percentage of Total Population', 'Year', 'Gender']].copy()

# new_table3_ES has only the values "WORLD", "Developed regions", "Developing regions", etc
values=["WORLD","Developed regions", "Developing regions", "Least developed countries", "Less developed regions excluding least developed countries", "Sub-Saharan Africa"]
new_table3_ES = table3_executive_summary[table3_executive_summary["Major area, region, country or area of destination"].isin(values)] 

new_table3_ES = new_table3_ES.reset_index(drop=True)

#Rename 
new_table3_ES = new_table3_ES.rename({'Major area, region, country or area of destination': 'Location'}, axis=1) 


# fix table 3 and remove all values in the values list
new_table3 = data_table3[~data_table3["Major area, region, country or area of destination"].isin(values)] 
new_table3 = new_table3.reset_index(drop=True)


#Problem: Major area, region, country or area of destination has multiple variables in the column and different variables in the rows. 
#tidy data principle #3: variables need to be in cells, not rows and columns
#Solution: Add a new column called "Category" that seperates the major area, region, country/area of destination 

# function to assign values in the category values of the new column 
def category(df):
    if df['Sort Order'] in list_major_area_num:
        return 'Major Area'
    elif df['Sort Order'] in list_region_num:
        return 'Region'
    return 'Country/Area'

new_table3 = new_table3.rename(columns={'Major area, region, country or area of destination': 'Location', 'Country code': 'Country Code'})


# change the floats type into ints
new_table3['Country Code'] = new_table3['Country Code'].astype(int)
new_table3['Sort Order'] = new_table3['Sort Order'].astype(int)

#Add category column
new_table3['Category'] = new_table3.apply(category, axis = 1)


#reorder the columns so that it follows order: sort order, country code..
new_table3 = new_table3[['Sort Order', 'Location', 'Category','Country Code', 'Notes', 'International Migrant Stock as a Percentage of Total Population', 'Year', 'Gender']]



#view
new_table3_ES.head()


Location Notes  \
0                                              WORLD   NaN   
1                                  Developed regions   (b)   
2                                 Developing regions   (c)   
3                          Least developed countries   (d)   
4  Less developed regions excluding least develop...   NaN   

   International Migrant Stock as a Percentage of Total Population  Year  \
0                                           2.873310                1990   
1                                           7.198015                1990   
2                                           1.685021                1990   
3                                           2.171513                1990   
4                                           1.617042                1990   

       Gender  
0  Both Sexes  
1  Both Sexes  
2  Both Sexes  
3  Both Sexes  
4  Both Sexes

In [16]:
##Cleaning Table 4##

#Read Table 4 on Excel without the first 14 rows
data_table4 = pd.read_excel('UN_MigrantStockTotal_2015.xlsx', 'Table 4', skiprows=14)

##Problem: The column names are values (years) and not variable names 
#tidy data principle #1: Column names need to be informative, variable names and not values

# The years are in the first row. To fix that problem, we have to change it into columns with their corresponding values. 
data_table4.columns = data_table4.iloc[0]

#Manually re-write the column names and organize based off of desired column names 
data_table4.columns = ['Sort order', 'Major area, region, country or area of destination', 'Notes', 'Country code', 'Type of data (a)',
'1990F', '1995F', '2000F', '2005F', '2010F', '2015F']


#Remove the first row                        
data_table4 = data_table4.iloc[1: , :]

#Create a column name called demographic that has the year+gender values

data_table4 = data_table4.melt(id_vars=['Sort order', 'Major area, region, country or area of destination', 'Notes', 'Country code', 'Type of data (a)'], var_name=["demographic"],value_name="Migrants as Percentage of the International Migrant Stock") 

##Problem: There are multiple variables (gender and year) stored in one column (demographic) 
#Tidy data principle 2: Each column needs to consist of one and only one variable
#Solution: Split the demographic column into gender and years 

data_table4 = (data_table4.assign(Year = lambda x: x.demographic.str[0:4].astype(str), Gender = lambda x: x.demographic.str[4:].astype(str)).drop("demographic",axis=1))

#Styling the dataset: Changing F females. Change Sort\norder column name to Sort Order

data_table4 = (data_table4.replace(to_replace =["F"],value =["Female"]))
data_table4 = data_table4.rename(columns={'Sort order': 'Sort Order'})    

#Drop the empty cells that contain value = .. 
data_table4['Migrants as Percentage of the International Migrant Stock'].replace('..', np.nan, inplace = True)  
data_table4.dropna(subset=['Migrants as Percentage of the International Migrant Stock'], inplace=True)

##Problem: The first 6 rows contain values whose country code does not exist in the annex. These rows also are sums of the other rows and do not belong under the same column. 
#Tidy data principle #4: each table column needs to have a singular data type 
#Solution: We created a seperate table that holds an executive summary of the Migrants as Percentage of the International Migrant Stock 

#Create a new dataframe containing desired columns

table4_executive_summary = data_table4[['Major area, region, country or area of destination', 'Notes', 'Migrants as Percentage of the International Migrant Stock', 'Year', 'Gender']].copy()

#table4_ES has only the values "WORLD", "Developed regions", "Developing regions", "Least developed countries", "Least developed regions", and "Sub-Saharan Africa"

values=["WORLD","Developed regions", "Developing regions", "Least developed countries", "Less developed regions excluding least developed countries", "Sub-Saharan Africa"]
table4_ES = table4_executive_summary[table4_executive_summary["Major area, region, country or area of destination"].isin(values)] 

table4_ES = table4_ES.reset_index(drop=True)

#Rename 
table4_ES = table4_ES.rename({'Major area, region, country or area of destination': 'Location'}, axis=1) 


#Fix table 1 and remove all values in the values list
new_table4 = data_table4[~data_table4["Major area, region, country or area of destination"].isin(values)] 
new_table4 = new_table4.reset_index(drop=True)


#Problem: Major area, region, country or area of destination has multiple variables in the column and different variables in the rows. 
#tidy data principle #3: variables need to be in cells, not rows and columns
#Solution: Add a new column called "Category" that seperates the major area, region, country/area of destination 

#Create list
list_major_area_num = [7, 71, 127, 180, 238]
list_region_num = [8, 29, 39, 47, 53, 72, 78, 86, 98, 108, 128, 139, 153, 170, 181, 208, 217, 232, 239, 242, 248, 256]

# function to assign values in the category values of the new column 
def category(df):
    if df['Sort Order'] in list_major_area_num:
        return 'Major Area'
    elif df['Sort Order'] in list_region_num:
        return 'Region'
    return 'Country/Area'

new_table4 = new_table4.rename(columns={'Major area, region, country or area of destination': 'Location', 'Country code': 'Country Code'})

# #change the float types into ints
new_table4['Sort Order'] = new_table4['Sort Order'].astype(int)
new_table4['Country Code'] = new_table4['Country Code'].astype(int)

# #Add category column
new_table4['Category'] = new_table4.apply(category, axis = 1)

# #reorder the columns so that it follows order: sort order, country code..
new_table4 = new_table4[['Sort Order', 'Location', 'Category','Country Code', 'Notes', 'Type of data (a)', 'Migrants as Percentage of the International Migrant Stock', 'Year', 'Gender']]

#view new_table4
table4_ES.head()



Location Notes  \
0                                              WORLD   NaN   
1                                  Developed regions   (b)   
2                                 Developing regions   (c)   
3                          Least developed countries   (d)   
4  Less developed regions excluding least develop...   NaN   

   Migrants as Percentage of the International Migrant Stock  Year  Gender  
0                                          49.039150          1990  Female  
1                                          51.123977          1990  Female  
2                                          46.592099          1990  Female  
3                                          47.261155          1990  Female  
4                                          46.466684          1990  Female

In [17]:
##Cleaning Table 5##

#Read Table 5 on Excel without the first 14 rows
data_table5 = pd.read_excel('UN_MigrantStockTotal_2015.xlsx', 'Table 5', skiprows=14)


##Problem: The column names are values (years) and not variable names 
#tidy data principle #1: Column names need to be informative, variable names and not values


# The years are in the first row. To fix that problem, we have to change it into columns with their corresponding values. 
data_table5.columns = data_table5.iloc[0]

# Manually re-write the column names and organize based off of desired names
data_table5.columns = ['Sort order', 'Major area, region, country or area of destination', 'Notes', 'Country code', 'Type of data (a)',
'1990-1995MF', '1995-2000MF', '2000-2005MF', '2005-2010MF', '2010-2015MF', '1990-1995M', '1995-2000M', '2000-2005M', '2005-2010M', '2010-2015M','1990-1995F', '1995-2000F', '2000-2005F', '2005-2010F', '2010-2015F']

#Remove the first row                        
data_table5 = data_table5.iloc[1: , :]

## Problem: The column names are values (years) and not variable names 
#Tidy data principle 1: Column names need to be informative, variable names and not values
#Solution: Create a column name called demographic that has the year+gender values
data_table5 = data_table5.melt(id_vars=['Sort order', 'Major area, region, country or area of destination', 'Notes', 'Country code', 'Type of data (a)'], var_name=["demographic"],value_name="Annual Rate of Change of the Migrant Stock") 

#Create a column name called demographic that has the year+gender values
data_table5=(data_table5.assign(Year = lambda x: x.demographic.str[0:9].astype(str), Gender = lambda x: x.demographic.str[9:].astype(str)).drop("demographic",axis=1))

#Styling the dataset: Changing MF to Both sexes, M to male, F to females. Change Sort Order column name 
data_table5 = (data_table5.replace(to_replace =["MF","M","F"],value =["Both Sexes","Male","Female"]))
data_table5 = data_table5.rename(columns={'Sort order': 'Sort Order'})        

#Drop the empty cells that contain value = .. 
data_table5['Annual Rate of Change of the Migrant Stock'].replace('..', np.nan, inplace = True)  
data_table5.dropna(subset=['Annual Rate of Change of the Migrant Stock'], inplace=True)

##Problem: The first 6 rows contain values whose country code does not exist in the annex. These rows also are sums of the other rows and do not belong under the same column. 
#Tidy data principle #4: each table column needs to have a singular data type 
#Solution: We created a seperate table that holds an executive summary of the Annual Rate of Change of the Migrant Stock

#Create a new dataframe containing desired columns
table5_executive_summary = data_table5[['Major area, region, country or area of destination', 'Notes', 'Annual Rate of Change of the Migrant Stock', 'Year', 'Gender']].copy()


# new_table5_ES has only the values "WORLD", "Developed regions", "Developing regions", etc
values=["WORLD","Developed regions", "Developing regions", "Least developed countries", "Less developed regions excluding least developed countries", "Sub-Saharan Africa"]
new_table5_ES = table5_executive_summary[table5_executive_summary["Major area, region, country or area of destination"].isin(values)] 

new_table5_ES = new_table5_ES.reset_index(drop=True)

#Rename 
new_table5_ES = new_table5_ES.rename({'Major area, region, country or area of destination': 'Location'}, axis=1) 


# fix table 5 and remove all values in the values list
new_table5 = data_table5[~data_table5["Major area, region, country or area of destination"].isin(values)] 
new_table5 = new_table5.reset_index(drop=True)

#Problem: Major area, region, country or area of destination has multiple variables in the column and different variables in the rows. 
#tidy data principle #3: variables need to be in cells, not rows and columns
#Solution: Add a new column called "Category" that seperates the major area, region, country/area of destination 

# function to assign values in the category values of the new column 
def category(df):
    if df['Sort Order'] in list_major_area_num:
        return 'Major Area'
    elif df['Sort Order'] in list_region_num:
        return 'Region'
    return 'Country/Area'

new_table5 = new_table5.rename(columns={'Major area, region, country or area of destination': 'Location', 'Country code': 'Country Code'})


# change the floats type into ints
new_table5['Country Code'] = new_table5['Country Code'].astype(int)
new_table5['Sort Order'] = new_table5['Sort Order'].astype(int)



#Add category column
new_table5['Category'] = new_table5.apply(category, axis = 1)


#reorder the columns so that it follows order: sort order, country code..
new_table5 = new_table5[['Sort Order', 'Location', 'Category','Country Code', 'Notes', 'Annual Rate of Change of the Migrant Stock', 'Year', 'Gender']]



#view
new_table5_ES.head()




Location Notes  \
0                                              WORLD   NaN   
1                                  Developed regions   (b)   
2                                 Developing regions   (c)   
3                          Least developed countries   (d)   
4  Less developed regions excluding least develop...   NaN   

   Annual Rate of Change of the Migrant Stock       Year      Gender  
0                                    1.051865  1990-1995  Both Sexes  
1                                    2.275847  1990-1995  Both Sexes  
2                                   -0.487389  1990-1995  Both Sexes  
3                                    1.118175  1990-1995  Both Sexes  
4                                   -0.803244  1990-1995  Both Sexes

In [ ]:
##Cleaning Table 6##

#Read Table 6 on Excel without the first 14 rows

data_table6 = pd.read_excel('UN_MigrantStockTotal_2015.xlsx', 'Table 6', skiprows=14)

##Problem: The column names are values (years) and not variable names 
#tidy data principle #1: Column names need to be informative, variable names and not values

# The years are in the first row. To fix that problem, we have to change it into columns with their corresponding values. 
data_table6.columns = data_table6.iloc[0]

#Manually re-write the column names and organize based off of desired column names 

data_table6.columns = ['Sort order', 'Major area, region, country or area of destination', 'Notes', 'Country code', 'Type of data (a)',
'1990MF', '1995MF', '2000MF', '2005MF', '2010MF', '2015MF', '1990', '1995', '2000', '2005', '2010', '2015', '1990-1995', '1995-2000', '2000-2005', '2005-2010', '2010-2015']

#Remove first row 

data_table6 = data_table6.iloc[1: , :]

#Styling the dataset: Changing MF to Both sexes, M to male, F to females. Change Sort\norder column name to Sort Order
data_table6 = data_table6.rename(columns={'Sort order': 'Sort Order'})    


#Problem: Major area, region, country or area of destination has multiple variables in the column and different variables in the rows. 
#tidy data principle #3: variables need to be in cells, not rows and columns
#Solution: Add a new column called "Category" that seperates the major area, region, country/area of destination 

#Create list
list_major_area_num = [7, 71, 127, 180, 238]
list_region_num = [8, 29, 39, 47, 53, 72, 78, 86, 98, 108, 128, 139, 153, 170, 181, 208, 217, 232, 239, 242, 248, 256]

# function to assign values in the category values of the new column 
def category(df):
    if df['Sort Order'] in list_major_area_num:
        return 'Major Area'
    elif df['Sort Order'] in list_region_num:
        return 'Region'
    return 'Country/Area'

new_table6 = data_table6.rename(columns={'Major area, region, country or area of destination': 'Location', 'Country code': 'Country Code'})

#change the float types into ints
new_table6['Sort Order'] = new_table6['Sort Order'].astype(int)
new_table6['Country Code'] = new_table6['Country Code'].astype(int)

#Add category column
new_table6['Category'] = new_table6.apply(category, axis = 1)

#reorder the columns so that it follows order: sort order, country code..
new_table6 = new_table6[['Sort Order', 'Location', 'Category','Country Code', 'Notes', 'Type of data (a)', '1990MF', '1995MF', '2000MF', '2005MF', '2010MF', '2015MF', '1990', '1995', '2000', '2005', '2010', '2015', '1990-1995', '1995-2000', '2000-2005', '2005-2010', '2010-2015']]



#Problem: There are multiple types of of data stored in 1 table
#tidy data principle #4: each table column needs to have a singular data type 
#Solution: Create three seperate tables - One for estimated refugee stock, one for refugees as percentage of migrant stock, and one for annual rate of change of the refugee stock


##Create a dataframe for Estimated Refugee stock at mid-year 
ERS_midyear_table = new_table6[['Sort Order', 'Location', 'Category', 'Country Code', 'Notes', 'Type of data (a)', '1990MF', '1995MF','2000MF', '2005MF', '2010MF', '2015MF']].copy()


##Problem: The column names are values (years) and not variable names 
#tidy data principle #1: Column names need to be informative, variable names and not values

#Create a column name called demographic that has the year+gender values

ERS_midyear_table = ERS_midyear_table.melt(id_vars=['Sort Order', 'Location', 'Category', 'Country Code', 'Notes', 'Type of data (a)'], var_name=["demographic"],value_name="Estimated Refugee Stock at Mid-Year") 

##Problem: There are multiple variables (gender and year) stored in one column (demographic) 
#Tidy data principle 2: Each column needs to consist of one and only one variable
#Solution: Split the demographic column into gender and years 

ERS_midyear_table = (ERS_midyear_table.assign(Year = lambda x: x.demographic.str[0:4].astype(str), Gender = lambda x: x.demographic.str[4:].astype(str)).drop("demographic",axis=1))

#Styling the dataset: Changing MF to Both sexes

ERS_midyear_table = (ERS_midyear_table.replace(to_replace =["MF"],value =["Both Sexes"]))

#Drop the empty cells that contain value = .. 

ERS_midyear_table['Estimated Refugee Stock at Mid-Year'].replace('..', np.nan, inplace = True)  
ERS_midyear_table.dropna(subset=['Estimated Refugee Stock at Mid-Year'], inplace=True)

##Problem: The first 6 rows contain values whose country code does not exist in the annex. These rows also are sums of the other rows and do not belong under the same column. 
#Tidy data principle #4: each table column needs to have a singular data type 
#Solution: We created a seperate table that holds an executive summary of the International Migrant Stock at Mid-Year

#Create a new dataframe containing desired columns

ERS_midyear_executive_summary = ERS_midyear_table[['Sort Order', 'Location', 'Notes', 'Type of data (a)','Estimated Refugee Stock at Mid-Year', 'Year', 'Gender']].copy()

#ERS_midyear_ES has only the values "WORLD", "Developed regions", "Developing regions", "Least developed countries", "Least developed regions", and "Sub-Saharan Africa"

values=["WORLD","Developed regions", "Developing regions", "Least developed countries", "Less developed regions excluding least developed countries", "Sub-Saharan Africa"]
ERS_midyear_ES = ERS_midyear_executive_summary[ERS_midyear_executive_summary["Location"].isin(values)] 

ERS_midyear_ES = ERS_midyear_ES.reset_index(drop=True)


#Fix ERS_midyear_table and remove all values in the values list
new_ERS_midyear_table = ERS_midyear_table[~ERS_midyear_table["Location"].isin(values)] 
new_ERS_midyear_table = new_ERS_midyear_table.reset_index(drop=True)


##Create a dataframe for Refugees as a percentage of the international migrant stock

RP_IMS_table = new_table6[['Sort Order', 'Location', 'Category', 'Country Code', 'Notes', 'Type of data (a)',  '1990', '1995', '2000', '2005', '2010', '2015']].copy()


##Problem: The column names are values (years) and not variable names 
#tidy data principle #1: Column names need to be informative, variable names and not values

#Create a column name called demographic that has the year+gender values

RP_IMS_table = RP_IMS_table.melt(id_vars=['Sort Order', 'Location', 'Category', 'Country Code', 'Notes', 'Type of data (a)'], var_name=["Year"],value_name="Refugees as a Percentage of the International Migrant Stock") 

#Drop the empty cells that contain value = .. 

RP_IMS_table['Refugees as a Percentage of the International Migrant Stock'].replace('..', np.nan, inplace = True)  
RP_IMS_table.dropna(subset=['Refugees as a Percentage of the International Migrant Stock'], inplace=True)


##Problem: The first 6 rows contain values whose country code does not exist in the annex. These rows also are sums of the other rows and do not belong under the same column. 
#Tidy data principle #4: each table column needs to have a singular data type 
#Solution: We created a seperate table that holds an executive summary of the International Migrant Stock at Mid-Year

#Create a new dataframe containing desired columns

RP_IMS_executive_summary = RP_IMS_table[['Sort Order', 'Location', 'Notes', 'Type of data (a)','Refugees as a Percentage of the International Migrant Stock']].copy()

#ERS_midyear_ES has only the values "WORLD", "Developed regions", "Developing regions", "Least developed countries", "Least developed regions", and "Sub-Saharan Africa"

values=["WORLD","Developed regions", "Developing regions", "Least developed countries", "Less developed regions excluding least developed countries", "Sub-Saharan Africa"]
RP_IMS_ES = RP_IMS_executive_summary[RP_IMS_executive_summary["Location"].isin(values)] 

RP_IMS_ES = RP_IMS_ES.reset_index(drop=True)



#Fix ERS_midyear_table and remove all values in the values list
new_RP_IMS_ES_table = RP_IMS_table[~RP_IMS_table["Location"].isin(values)] 
new_RP_IMS_ES_table = new_RP_IMS_ES_table.reset_index(drop=True)

                                  
##Create a dataframe for Annual Rate of Change of the Refugee Stock 
ARC_RS_table = new_table6[['Sort Order', 'Location', 'Category', 'Country Code', 'Notes', 'Type of data (a)', '1990-1995', '1995-2000', '2000-2005', '2005-2010', '2010-2015']].copy()


##Problem: The column names are values (years) and not variable names 
#tidy data principle #1: Column names need to be informative, variable names and not values

#Create a column name called demographic that has the year+gender values

ARC_RS_table = ARC_RS_table.melt(id_vars=['Sort Order', 'Location', 'Category', 'Country Code', 'Notes', 'Type of data (a)'], var_name=["Year"],value_name="Annual Rate of Change of the Refugee Stock") 

#Drop the empty cells that contain value = .. 

ARC_RS_table['Annual Rate of Change of the Refugee Stock'].replace('..', np.nan, inplace = True)  
ARC_RS_table.dropna(subset=['Annual Rate of Change of the Refugee Stock'], inplace=True)


##Problem: The first 6 rows contain values whose country code does not exist in the annex. These rows also are sums of the other rows and do not belong under the same column. 
#Tidy data principle #4: each table column needs to have a singular data type 
#Solution: We created a seperate table that holds an executive summary of the Annual Rate of Change of the Refugee Stock

#Create a new dataframe containing desired columns

ARC_RS_executive_summary = ARC_RS_table[['Sort Order', 'Location', 'Notes', 'Type of data (a)','Annual Rate of Change of the Refugee Stock']].copy()

#ERS_midyear_ES has only the values "WORLD", "Developed regions", "Developing regions", "Least developed countries", "Least developed regions", and "Sub-Saharan Africa"

values=["WORLD","Developed regions", "Developing regions", "Least developed countries", "Less developed regions excluding least developed countries", "Sub-Saharan Africa"]
ARC_RS_ES = ARC_RS_executive_summary[ARC_RS_executive_summary["Location"].isin(values)] 

ARC_RS_ES = ARC_RS_ES.reset_index(drop=True)
                                                                  

#Fix ERS_midyear_table and remove all values in the values list
new_ARC_RS_table = ARC_RS_table[~ARC_RS_table["Location"].isin(values)] 
new_ARC_RS_table = new_ARC_RS_table.reset_index(drop=True)


#view
ERS_midyear_table.head()